In [71]:
import pandas as pd
import sklearn.svm as svm
import sklearn.metrics as mt
import numpy as np

from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier

# 1. 데이터 로드

In [72]:
Data= pd.read_csv("C:/Users/19lya/Documents/Python/Data/ML TeamProj/master.csv")

In [73]:
MiniData=pd.read_csv("C:/Users/19lya/Documents/Python/Data/ML TeamProj/master_mini.csv")

In [74]:
# Data.info

In [75]:
# MiniData.info

# 2. 변수 제거 후 X데이터,Y데이터로 나누기

In [76]:
lz= list(Data.columns)

lz.remove('outcome_critical')
lz.remove('intime')
lz.remove('outtime')
lz.remove('ed_los')
lz.remove('outcome_icu_transfer_12h')

In [77]:
X= Data[lz]
Y= Data[['outcome_critical']]

In [78]:
lz= list(MiniData.columns)

lz.remove('outcome_critical')
lz.remove('intime')
lz.remove('outtime')
lz.remove('ed_los')
lz.remove('outcome_icu_transfer_12h')

In [79]:
X_mini= MiniData[lz]
Y_mini= MiniData[['outcome_critical']]

# 3. Training데이터, Test데이터로 나누기

In [80]:
#Imbalanced classification-> SMOTE
sm = SMOTE(random_state=0)
X_smote, Y_smote = sm.fit_sample(X, Y)

In [81]:
sm = SMOTE(random_state=0) 
X_smote_mini, Y_smote_mini = sm.fit_sample(X_mini, Y_mini)

In [82]:
X_train, X_test, Y_train, Y_test= train_test_split(X_smote, Y_smote, test_size= 0.25) 

In [83]:
X_train_mini, X_test_mini, Y_train_mini, Y_test_mini= train_test_split(X_smote_mini, Y_smote_mini, test_size= 0.25)

In [84]:
X_train.shape

(629080, 96)

In [85]:
X_test.shape

(209694, 96)

In [86]:
X_train_mini.shape

(6318, 98)

In [87]:
X_test_mini.shape

(2106, 98)

# 4. SVM 모델 생성, 학습

1. linear kernel SVM -> Too high time complexity

In [88]:
# clf= svm.SVC(kernel= 'linear')
# clf.fit(X_train, Y_train)
# Y_pred= clf.predict(X_test)

In [89]:
# clf.get_params()

2. rbf kernel SVM -> Too high time complexity

In [90]:
# clf2= svm.SVC(kernel= 'rbf', C= 8, gamma= .1) 
# #C: 정규화비용(크면 좁은 margin, 작으면 넓은 margin)
# #gamma: 보통 1/sigma^2, 크면 예민한db, 작으면 smooth한db

# clf2.fit(X_train, Y_train)
# Y_pred2= clf2.predict(X_test)

In [91]:
# clf2.get_params()

3. SGDClassifier 사용

In [92]:
sgdClf= SGDClassifier(loss='hinge', learning_rate= 'optimal', epsilon= .05, alpha= .01, random_state= 2001)
sgdClf.fit(X_train, Y_train)
Y_pred3= sgdClf.predict(X_test)

C:\Users\19lya\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\19lya\anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:570: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "


loss: 확률적 경사하강법으로 최적화할 손실함수 지정 <br/>
penalty: 규제 종류 지정 (기본: 'l2') <br/>
alpha: 규제 강도 <br/>
max_iter: 에포크 횟수 <br/>
tol: 반복 멈출 조건 (기본: .001)

In [93]:
sgdClf_mini= SGDClassifier(loss='hinge', learning_rate= 'optimal', epsilon= .05, alpha= .01, random_state= 2001)
sgdClf_mini.fit(X_train_mini, Y_train_mini)
Y_pred_mini= sgdClf_mini.predict(X_test_mini)

C:\Users\19lya\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


# 5. 모델 평가하기

In [94]:
# print("Accuracy: ", mt.accuracy_score(Y_test, Y_pred))
# print("Precision: ", mt.precision_score(Y_test, Y_pred))
# print("Recall: ", mt.recall_score(Y_test, Y_pred))

In [95]:
# print("="*100)

In [96]:
# print("Accuracy: ", mt.accuracy_score(Y_test, Y_pred2))
# print("Precision: ", mt.precision_score(Y_test, Y_pred2))
# print("Recall: ", mt.recall_score(Y_test, Y_pred2))

In [97]:
# print("="*100)

In [98]:
print("Accuracy: ", mt.accuracy_score(Y_test, Y_pred3))
print("Precision: ", mt.precision_score(Y_test, Y_pred3, average='weighted', labels=np.unique(Y_pred3)))
print("Recall: ", mt.recall_score(Y_test, Y_pred3, average='weighted', labels=np.unique(Y_pred3)))

print("="*50)

print(classification_report(Y_test, Y_pred3, zero_division=1))

Accuracy:  0.49943727526777115
Precision:  0.4994372752677712
Recall:  1.0
              precision    recall  f1-score   support

       False       1.00      0.00      0.00    104965
        True       0.50      1.00      0.67    104729

    accuracy                           0.50    209694
   macro avg       0.75      0.50      0.33    209694
weighted avg       0.75      0.50      0.33    209694



In [99]:
print("Accuracy: ", mt.accuracy_score(Y_test_mini, Y_pred_mini))
print("Precision: ", mt.precision_score(Y_test_mini, Y_pred_mini, zero_division=1, average='weighted'))
print("Recall: ", mt.recall_score(Y_test_mini, Y_pred_mini,zero_division=1, average='weighted'))

print("="*50)

print(classification_report(Y_test_mini, Y_pred_mini, zero_division= 1))

Accuracy:  0.49477682811016144
Precision:  0.7500272815245907
Recall:  0.49477682811016144
              precision    recall  f1-score   support

       False       0.49      1.00      0.66      1042
        True       1.00      0.00      0.00      1064

    accuracy                           0.49      2106
   macro avg       0.75      0.50      0.33      2106
weighted avg       0.75      0.49      0.33      2106



# 6. 하이퍼파리미터 튜닝

Kernel: The main function of the kernel is to transform the given dataset input data into the required form. There are various types of functions such as linear, polynomial, and radial basis function (RBF). Polynomial and RBF are useful for non-linear hyperplane. Polynomial and RBF kernels compute the separation line in the higher dimension. In some of the applications, it is suggested to use a more complex kernel to separate the classes that are curved or nonlinear. This transformation can lead to more accurate classifiers.

Regularization: Regularization parameter in python's Scikit-learn C parameter used to maintain regularization. Here C is the penalty parameter, which represents misclassification or error term. The misclassification or error term tells the SVM optimization how much error is bearable. This is how you can control the trade-off between decision boundary and misclassification term. A smaller value of C creates a small-margin hyperplane and a larger value of C creates a larger-margin hyperplane.

Gamma: A lower value of Gamma will loosely fit the training dataset, whereas a higher value of gamma will exactly fit the training dataset, which causes over-fitting. In other words, you can say a low value of gamma considers only nearby points in calculating the separation line, while the a value of gamma considers all the data points in the calculation of the separation line.

In [100]:
# sgdClf.get_params()

In [101]:
# parameters = {'alpha': (1e-2, 1e-3)}
 
# sgdClf_gs = GridSearchCV(sgdClf, parameters)
# sgdClf_gs.fit(X_train, Y_train)

In [102]:
# Y_predGS= sgdClf_gs.predict(X_test)
# print(classification_report(Y_test, Y_predGS))

In [103]:
# parameters = {'alpha': (1e-2, 1e-3)}

# sgdClf_gs_mini = GridSearchCV(sgdClf_mini, parameters)
# sgdClf_gs_mini.fit(X_train_mini, Y_train_mini)

In [104]:
# Y_predGS_mini= sgdClf_gs_mini.predict(X_test_mini)
# print(classification_report(Y_test_mini, Y_predGS_mini))

# 7. 시각화

In [105]:
#변수 개수 2개로 줄여야 graph에 시각화 가능
'''
plot_decision_regions(X_combined_std, 
                      y_combined,
                      classifier=svm, 
                      test_idx=range(105, 150))
plt.scatter(svm.dual_coef_[0, :], svm.dual_coef_[1, :])
plt.xlabel('petal length [standardized]')
plt.ylabel('petal width [standardized]')
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()
'''

"\nplot_decision_regions(X_combined_std, \n                      y_combined,\n                      classifier=svm, \n                      test_idx=range(105, 150))\nplt.scatter(svm.dual_coef_[0, :], svm.dual_coef_[1, :])\nplt.xlabel('petal length [standardized]')\nplt.ylabel('petal width [standardized]')\nplt.legend(loc='upper left')\nplt.tight_layout()\nplt.show()\n"